# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [2]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())
# # 주소 이동
# url = 'https://map.naver.com/'
# driver.get(url)
# time.sleep(1)


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

In [3]:
new_df_200 = pd.read_csv('new_df_200.csv')

In [4]:
new_df = pd.DataFrame()
restaurant_name_list = []
category_name_list = []
address_list = []
menu_list = []
price_list = []

In [5]:
## 인덱스 초기화 시켜줘야 돌아감

new_df_200.reset_index(inplace=True)
new_df_200.drop('index',axis=1,inplace=True)
new_df_200.head()

,Unnamed: 0,level_0,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,0,0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,1,1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
2,2,3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
3,3,5,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로204,",가능
4,4,7,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길11,",가능


In [6]:
new_df_200.drop(['Unnamed: 0','level_0'],axis=1,inplace=True)

In [7]:
new_df_200.loc[new_df_200['사업장명']=='프로티너(PROTEINER) 강남역점',:]

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
103,영업/정상,서울특별시 강남구 역삼동 831-24 예미프레스티지,"서울특별시 강남구 강남대로78길 12, 예미프레스티지 지상1층 101호 중 102호...",프로티너(PROTEINER) 강남역점,경양식,202614.86159,443629.023396,서울특별시,강남구,"강남대로78길12,",가능


In [11]:
import re

def remove_parentheses(input_string):
    start_index = input_string.find('(')
    end_index = input_string.find(')')
    
    if start_index != -1 and end_index != -1:  # '('와 ')'가 모두 존재하는 경우
        return input_string[:start_index] + input_string[end_index+1:]
    else:
        return input_string

# 테스트
input_strings = ['만소당', '만소당(Mansodang)', '프로티너(PROTEINER) 강남역점']
for input_string in input_strings:
    result = remove_parentheses(input_string)
    print(result)

만소당
만소당
프로티너 강남역점


In [13]:
## 그냥 식당이름이면 식당이름 , 식당이름 () + ~역점이면 식당이름 + ~역점, 식당이름 () 면 식당이름만 나오게 
def remove_parentheses_from_column(df, column_name):
    def remove_parentheses(input_string):
        start_index = input_string.find('(')
        end_index = input_string.find(')')
        
        if start_index != -1 and end_index != -1:  # '('와 ')'가 모두 존재하는 경우
            return input_string[:start_index] + input_string[end_index+1:]
        else:
            return input_string
    
    df[column_name] = df[column_name].apply(remove_parentheses)
    return df



        Name
0        만소당
1        만소당
2  프로티너 강남역점


In [14]:
remove_parentheses_from_column(new_df_200, '사업장명')

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
2,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
3,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로204,",가능
4,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길11,",가능
...,...,...,...,...,...,...,...,...,...,...,...
133,영업/정상,서울특별시 강남구 역삼동 707-34 한신인터밸리24,"서울특별시 강남구 테헤란로 322, 한신인터밸리24 지하2층 제비145호 (역삼동)",캐주얼 샐러드,기타,204044.789949,444529.840427,서울특별시,강남구,"테헤란로322,",가능
134,영업/정상,서울특별시 강남구 역삼동 738-43 삼화빌딩,"서울특별시 강남구 논현로81길 13, 삼화빌딩 지상1층 (역삼동)",SPEVO 스페보,경양식,203188.832405,443960.976473,서울특별시,강남구,"논현로81길13,",가능
135,영업/정상,서울특별시 강남구 개포동 1188,"서울특별시 강남구 논현로8길 12, 지상1층 101호 (개포동)",13월,한식,204359.091172,441290.315272,서울특별시,강남구,"논현로8길12,",가능
136,영업/정상,서울특별시 강남구 논현동 114-20,"서울특별시 강남구 언주로134길 11, 지상1층 (논현동)",몽이네 한우내장탕,한식,203159.048808,446075.348000,서울특별시,강남구,"언주로134길11,",가능


In [16]:
new_df_200['사업장명'].unique()

array(['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지', '논현136', '피양옥', '세븐어클락', '김재윤김치', '앰비언트 피크닉',
       '쓰촨 마라', '스시 쥬고야', '조선육회', '브레드_히어_', '후토루 역삼점', '블루플레임',
       '왕손피자 역삼점', '몽크스부처', '난바스낵 압구정로데오', '국신라면', '킹도리탕', '라이스박스 논현점',
       '포시애틀 은마상가점', '다이어메이커', 'ETHOS 압구정', '가치있는 샐러드', '팀 홀튼 신논현역점',
       '옥된장 도곡점', '토종돼지 삼겹살전문', '압구정육식당', '마농 에 마농 마농와인', '일월분식', '그라도스',
       '일리있는', '갸또올리비에', '더블유', '라장양갈비·양꼬치 무한리필', '경복', '한신 김치찌개',
       '치치 다이닝', '쇼미비치', '노슈가존', '도산포차', '도리토리 삼성직영점', '아맘돈까스 한티역점',
       '맘스피자 대치사거리점', '플로우라운지', '아이언케이스', '이목 스모크다이닝', '느린마을양조장 압구정로데오점',
       '진스토랑', '영희탕후루', '호시타코야끼 대치점', '에그앤플라워', '바르다김선생 압구정역점', '모티에',
       '솔트 플러스 39', '청담 수', '타이거꼬마김밥', '로쿄 신논현', '도산정육블랙 청담본점', '신의주찹쌀순대',
       '이더웨이 쏘 굿', '초이다이닝 강남점', '랩', '산월수제비', '도란', '금돈옥 청담점',
       '엠브이피자 강남점', '앨리스 스튜디오 공간대여 티카페 강남점', '데일리픽스', '헤일리스커피', '부리또리',
       '청담

# 문제 1
개포로504, 37.5 시그니처 개포점 -> 전체 데이터 돌릴떈 안돌아가는데 하나만 돌릴떈 돌아감, 그렇다고해서 검색해서 바로 안넘어가는 케이스도 아님

-> 이거 말고 더 있을 수 있음
# 문제 2

만소당 -> test 데이터로 돌릴떈 프레임 넘어가고 정보도 잘 불러옴 근데 전체로 할떈 안불러옴 

-> 이거 말고 더 있을 수 있음 

--> 기다리는 시간을 바꿔보자 

# 문제3
세븐어클락이라는 식당 --> 테헤란로6길30, 세븐어클락(7O'CLOCK) 이라고 검색해서 검색 완료됌.

                      --> 근데 식당정보 크롤링할때 동일하게 검색했는데 안뜸

                      --> 근데 세븐어클락이라고 치면 결과 나옴, 주소도 같음. 

                      --> 테헤란로6길30, 세븐어클락 이라고 쳐도 나옴 ;; (7O'CLOCK) 포함해서 검색하면 안됌 

-> 이거는 그냥 노가다 떄려야할것같스빈ㄷ.
                      





In [9]:
new_df_200['사업장명'].unique()

array(['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",
       '김재윤김치', '앰비언트 피크닉(AMBIENT PICNIK)', '쓰촨 마라', '스시 쥬고야', '조선육회',
       '브레드_히어_(Bread_Here_)', '후토루 역삼점', '블루플레임', '왕손피자 역삼점', '몽크스부처',
       '난바스낵 압구정로데오', '국신라면', '킹도리탕', '라이스박스 논현점', '포시애틀 은마상가점',
       '다이어메이커(Dyer Maker)', 'ETHOS 압구정', '가치있는 샐러드', '팀 홀튼 신논현역점',
       '옥된장 도곡점', '토종돼지 삼겹살전문', '압구정육식당', '마농 에 마농 마농와인', '일월분식', '그라도스',
       '일리있는', '갸또올리비에', '더블유(W)', '라장양갈비·양꼬치 무한리필', '경복', '한신 김치찌개',
       '치치 다이닝', '쇼미비치', '노슈가존', '도산포차', '도리토리 삼성직영점', '아맘돈까스 한티역점',
       '맘스피자 대치사거리점', '플로우라운지(FLOW LOUNGE)', '아이언케이스', '이목 스모크다이닝',
       '느린마을양조장 압구정로데오점', "진스토랑(Jin's torang)", '영희탕후루', '호시타코야끼 대치점',
       '에그앤플라워', '바르다김선생 압구정역점', '모티에(MOTIE)', '솔트 플러스 39(Salt Plus 39)',
       '청담 수', '타이거꼬마김밥(언주역점)', '로쿄 신논현', '도산정육블랙 청담본점', '신의주찹쌀순대',
       '이더웨이 쏘 굿

In [10]:
new_df_200.iloc[10,:]

영업상태명                                          영업/정상
소재지전체주소                   서울특별시 강남구 삼성동 144-14 채널리저브
도로명전체주소    서울특별시 강남구 삼성로 517, 지상1층 104호 (삼성동, 채널리저브)
사업장명                                        냉동고집 삼성점
업태구분명                                             한식
좌표정보(x)                                 204828.18035
좌표정보(y)                                445089.672895
시도                                             서울특별시
지역구                                              강남구
상세주소                                         삼성로517,
검색여부                                              가능
Name: 10, dtype: object

In [11]:
test_df = new_df_200.iloc[10:105,:]
test_df.reset_index(inplace=True)

In [12]:
# 일단 검색결과 안나오고 바로 페이지가 넘어간다고 가정하고 

for i in range(0,len(test_df)):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = test_df['상세주소'][i] + ' ' + test_df['사업장명'][i]
    # name = '논현로159길10,' + ' ' + '만소당	'
    driver.get('https://map.naver.com/p/search/{}'.format(name))

        
    time.sleep(3)  

    ## 화면이 안넘어가고 검색결과에 머물러 있을 때는 entryIframe 이 없음
    ## 그래서 검색어 쳤을때 entryIframe 있으면 --> 화면이 넘어갔다는 뜻이고 
    ## 없으면 아직 안넘어간 뜻이니깐 searchIframe 에서 클릭해서 넘어가야함 .
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
            
            time.sleep(3) 
            # 가게 이름, 변수로 지정
            restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
            category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
            address = driver.find_element(By.CLASS_NAME,'LDgIH').text
            if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                price = driver.find_element(By.CLASS_NAME,'mkBm3').text
            elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                price = driver.find_element(By.CLASS_NAME,'Yrsei').text
            else : 
                menu = '메뉴없음'
                price = "가격없음"
            
        else : 
            # searchIframe 프레임 

                searchIframe = driver.find_element(By.ID,'searchIframe')
                driver.switch_to.frame(searchIframe)


                ## 클릭 
                driver.find_element(By.CLASS_NAME,'YwYLL').click()
                
                # 기본프렝미 
                time.sleep(1)
                driver.switch_to.default_content()
            

                # entryIframe 프레임 
                entryIframe = driver.find_element(By.ID,'entryIframe')
                driver.switch_to.frame(entryIframe)

                time.sleep(3) 
                # 가게 이름, 변수로 지정
                restaurant_name = driver.find_element(By.CLASS_NAME,'Fc1rA').text
                category_name = driver.find_element(By.CLASS_NAME,'DJJvD').text
                address = driver.find_element(By.CLASS_NAME,'LDgIH').text
                if driver.find_elements(By.CLASS_NAME,'ihmWt') :
                    menu = driver.find_element(By.CLASS_NAME,'ihmWt').text
                    price = driver.find_element(By.CLASS_NAME,'mkBm3').text
                elif driver.find_elements(By.CLASS_NAME,'VQvNX') :
                    menu = driver.find_element(By.CLASS_NAME,'VQvNX').text
                    price = driver.find_element(By.CLASS_NAME,'Yrsei').text
                else : 
                    menu = '메뉴없음'
                    price = "가격없음"
    except Exception as e : 
        restaurant_name = name 
        category_name = '전처리필요'
        address = '전처리필요'
        menu = '메뉴없음'
        price = "가격없음"
        print(f"에러 메시지: {str(e)}")


    ## 리스트 ㄱ 
    restaurant_name_list.append(restaurant_name)
    category_name_list.append(category_name)
    address_list.append(address)
    menu_list.append(menu)
    price_list.append(price)
    
    print('식당 이름:',restaurant_name)
    print('식당 카테고리:',category_name)
    print('주소:',address)
    print('대표 메뉴:',menu)
    print('가격:',price)
    print('-----------------------------')
naver_df_200 = pd.DataFrame({'식당이름':restaurant_name_list ,
                             '업태구분' :category_name_list ,
                             '주소' :address_list ,
                             '메뉴': menu_list,
                             '가격':price_list
                             })

    

식당 이름: 냉동고집 삼성점
식당 카테고리: 육류,고기요리
주소: 서울 강남구 삼성로 517 1층 104호
대표 메뉴: 냉동삼겹살
가격: 15,000원
-----------------------------
식당 이름: 리하베스트
식당 카테고리: 카페,디저트
주소: 서울 강남구 역삼로 220 1층
대표 메뉴: 메뉴없음
가격: 가격없음
-----------------------------
식당 이름: 팡뮤제 베이커리
식당 카테고리: 베이커리
주소: 경기 김포시 풍무로 128 웰라움퍼펙트시티 1층
대표 메뉴: 아메리카노
가격: 4,800원
-----------------------------
식당 이름: 만소당
식당 카테고리: 돈가스
주소: 서울 강남구 논현로159길 10 104호
대표 메뉴: 돈까스
가격: 13,000원
-----------------------------
식당 이름: 꼬모 서울
식당 카테고리: 카페,디저트
주소: 서울 강남구 봉은사로74길 19 1층
대표 메뉴: 이탈리안 티라미수
가격: 6,500원
-----------------------------
식당 이름: 살롱 드 아난티 겐지
식당 카테고리: 일식당
주소: 서울 강남구 논현로136길 11 3층
대표 메뉴: (런치) 오반자이 테이쇼쿠
가격: 39,000원
-----------------------------
식당 이름: 논현136
식당 카테고리: 한식
주소: 서울 강남구 논현로136길 5-5 1층
대표 메뉴: 점심뷔페
가격: 13,000원
-----------------------------
식당 이름: 피양옥
식당 카테고리: 냉면
주소: 서울 강남구 압구정로36길 7 지상 1층, 2층
대표 메뉴: 만두
가격: 14,000원
-----------------------------
에러 메시지: Message: no such element: Unable to locate element: {"method":"css selector","selector":".YwYLL"}
  (Session inf

['정식당', '채선당 샤브보트 강남역점', '수아당', '알찬한끼', '세계관', '분식을 품다', '식당 민홍',
       '제주그집', '만수정 강남점', '은서네 국내산 대패삼겹살', '냉동고집 삼성점', '리하베스트', '팡뮤제',
       '만소당', '꼬모 서울', '겐지(KENJI)', '논현136', '피양옥', "세븐어클락(7O'CLOCK)",

In [14]:
test_df.loc[test_df['사업장명']=='일월분식',:]

,index,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
33,43,영업/정상,서울특별시 강남구 역삼동 858 강남역,"서울특별시 강남구 강남대로 지하 396, 강남역 지하1층 F-6호 (역삼동)",일월분식,분식,202422.25174,443872.484453,서울특별시,강남구,강남대로지하,가능


In [13]:
naver_df_200.head()


,식당이름,업태구분,주소,메뉴,가격
0,냉동고집 삼성점,"육류,고기요리",서울 강남구 삼성로 517 1층 104호,냉동삼겹살,"15,000원"
1,리하베스트,"카페,디저트",서울 강남구 역삼로 220 1층,메뉴없음,가격없음
2,팡뮤제 베이커리,베이커리,경기 김포시 풍무로 128 웰라움퍼펙트시티 1층,아메리카노,"4,800원"
3,만소당,돈가스,서울 강남구 논현로159길 10 104호,돈까스,"13,000원"
4,꼬모 서울,"카페,디저트",서울 강남구 봉은사로74길 19 1층,이탈리안 티라미수,"6,500원"


In [ ]:
naver_df

,식당이름,업태구분,주소,메뉴,가격
0,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
1,정식당,한식,서울 강남구 강남대로114길 18 . 1층 정식당,소불고기주물럭,"16,000원"
2,채선당 샤브보트 강남역점,샤브샤브,서울 강남구 강남대로 378 . 지상1층,소고기 샤브샤브,"12,900원"
3,수아당 갤러리아백화점 명품관 WEST점,김밥,서울 강남구 압구정로 343,유부월남쌈,"18,000원"
4,알찬한끼,종합분식,서울 강남구 역삼로 204 삼환아르누보 지하1층 107호,알찬도시락,"5,000원"
5,세계관 삼성본점,한식,서울 강남구 삼성로96길 11 2층,투뿔 한우 곱도리탕,"32,000원"
6,분식을품다 한티점,종합분식,서울 강남구 역삼로 409 1층 102호,쌀 떡볶이,"5,900원"
7,식당민홍,일식당,서울 강남구 논현로146길 11 2층,오마카세 코스,"122,000원"
8,제주그집,한식,서울 강남구 논현로8길 11 지상1층,냉동삼겹살(150g),"14,000원"
9,만수정 강남점,"장어,먹장어요리",서울 강남구 강남대로 628 비율라빌딩 1층,민물장어 1kg ( 650~680g),"90,000원"


# 카카오맵 별점 가져오기

In [ ]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')


# 카카오 별점 가져오는 함수 
def kakao_finder(place,name) :
    # 검색어 = 지역명 + 가게명 ex) 강남구 떡도리탕
    kakao_search_term = place + " " + name

    # 검색 결과가 있을 때
    try :               
        time.sleep(3)
        WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.ID,'search.keyword.query')))
        # 검색 박스 비우고 검색어 입력하고 enter
        driver.find_element(By.ID,'search.keyword.query').clear()
        driver.find_element(By.ID,'search.keyword.query').send_keys(kakao_search_term)
        driver.find_element(By.ID,'search.keyword.query').send_keys(Keys.RETURN)

        # 2초 대기
        time.sleep(2)

        # 별점 저장
        kakao_star = driver.find_element(By.XPATH,'//*[@id="info.search.place.list"]/li[1]/div[4]/span[1]/em').text
        kakao_star = float(kakao_star)
        
    # 검색 결과가 없을 때
    except :
        kakao_star = '없음' 
    
    return kakao_star


# test 
url = 'https://map.kakao.com/'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

지역구 = '강남구'
매장명 = ['맥도날드','호랑가시','떡도리탕']
star = {}

for i in range(len(매장명)) :
    star[매장명[i]] = kakao_finder(지역구,매장명[i])

star